In [1]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
# подключаем библиотеку для построения линейной регрессии
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import PolynomialFeatures
from sklearn.linear_model import Lasso, Ridge

In [51]:
#загружаем набор данных и выводим 5 первых строк
bunch = pd.read_csv("forestfires.csv")
bunch.head()

,X,Y,month,day,FFMC,DMC,DC,ISI,temp,RH,wind,rain,area
0,7,5,mar,fri,86.2,26.2,94.3,5.1,8.2,51,6.7,0.0,0.0
1,7,4,oct,tue,90.6,35.4,669.1,6.7,18.0,33,0.9,0.0,0.0
2,7,4,oct,sat,90.6,43.7,686.9,6.7,14.6,33,1.3,0.0,0.0
3,8,6,mar,fri,91.7,33.3,77.5,9.0,8.3,97,4.0,0.2,0.0
4,8,6,mar,sun,89.3,51.3,102.2,9.6,11.4,99,1.8,0.0,0.0


In [52]:
#проверяем числог строк и столбцов в наборе данных
bunch.shape

(517, 13)

In [53]:
#проверяем заполнение строк и типы признаков
bunch.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 517 entries, 0 to 516
Data columns (total 13 columns):
X        517 non-null int64
Y        517 non-null int64
month    517 non-null object
day      517 non-null object
FFMC     517 non-null float64
DMC      517 non-null float64
DC       517 non-null float64
ISI      517 non-null float64
temp     517 non-null float64
RH       517 non-null int64
wind     517 non-null float64
rain     517 non-null float64
area     517 non-null float64
dtypes: float64(8), int64(3), object(2)
memory usage: 52.6+ KB


In [54]:
# кодируем нечисловые столбцы методом дамми-кодирования
bunch = pd.concat([bunch, 
                      pd.get_dummies(bunch['month'], prefix="month"),
                      pd.get_dummies(bunch['day'], prefix="day"),],
                     axis=1)

In [55]:
#проверяем заполнение строк и типы признаков
bunch.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 517 entries, 0 to 516
Data columns (total 32 columns):
X            517 non-null int64
Y            517 non-null int64
month        517 non-null object
day          517 non-null object
FFMC         517 non-null float64
DMC          517 non-null float64
DC           517 non-null float64
ISI          517 non-null float64
temp         517 non-null float64
RH           517 non-null int64
wind         517 non-null float64
rain         517 non-null float64
area         517 non-null float64
month_apr    517 non-null uint8
month_aug    517 non-null uint8
month_dec    517 non-null uint8
month_feb    517 non-null uint8
month_jan    517 non-null uint8
month_jul    517 non-null uint8
month_jun    517 non-null uint8
month_mar    517 non-null uint8
month_may    517 non-null uint8
month_nov    517 non-null uint8
month_oct    517 non-null uint8
month_sep    517 non-null uint8
day_fri      517 non-null uint8
day_mon      517 non-null uint8
day_sat      

In [56]:
#указываем входы и выходы модели
X = bunch.drop(['area','month', 'day'], axis=1) 
#выбрасываем столбец area из входа; axis если =0 - строка, если = 1 -столбец
#ID выбросили, потому что не влияет на результаты предсказания
y = bunch['area']

In [57]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=11)

In [58]:
X_train.shape, X_test.shape

((413, 29), (104, 29))

In [59]:
lr = LinearRegression()
lr.fit(X_train, y_train)
lr.score(X_test, y_test) # считаем коэффициент детерминации

-0.024536262772802072

In [33]:
#среднеквадратическая ошибка
mean_squared_error(y_test, lr.predict(X_test))

6632.978958414131

In [34]:
#выводим коэффициенты b и k из уравнения y=kx+b; в лекциях b - w0, k - w1
lr.intercept_, lr.coef_

(4.35633538006897,
 array([  1.53792322,   0.52339679,   0.27589558,   0.18254454,
         -0.14147781,  -0.73500935,   0.81103173,  -0.20086159,
          1.21473873,  -0.88185508, -11.70835052,  13.55005543,
         11.00385754, -15.8340095 ,  10.89725519,   0.51511784,
         -6.33622993, -16.72438263, -20.44350642, -16.15988738,
         23.13673752,  28.10334285,  -2.20746592,  -2.16512732,
          9.26650612,  -0.62737163,  -3.55148259,   0.88939043,
         -1.60444909, -11.70835052,  13.55005543,  11.00385754,
        -15.8340095 ,  10.89725519,   0.51511784,  -6.33622993,
        -16.72438263, -20.44350642, -16.15988738,  23.13673752,
         28.10334285,  -2.20746592,  -2.16512732,   9.26650612,
         -0.62737163,  -3.55148259,   0.88939043,  -1.60444909]))

In [66]:
#проверяем, нельзя ли улучшить линейную регрессию, добавив полиномиальные признаки
pf = PolynomialFeatures(degree=3)
X_train_polynomial = pf.fit_transform(X_train)
X_test_polynomial = pf.fit_transform(X_test)
lf = LinearRegression()
lf.fit(X_train_polynomial, y_train)
lf.score(X_train_polynomial, y_train) #смотрим R^2 на обучающей выборке

0.9998770430097306

In [67]:
#смотрим R^2 на тестовой выборке
lf.score(X_test_polynomial, y_test)

-22427.37634470238

In [82]:
from sklearn.linear_model import Lasso, Ridge
lf = Ridge(fit_intercept=True, alpha=2, normalize=True)
lf.fit(X_train_polynomial, y_train)

Ridge(alpha=2, copy_X=True, fit_intercept=True, max_iter=None, normalize=True,
   random_state=None, solver='auto', tol=0.001)

In [83]:
lf.score(X_test_polynomial, y_test)

-0.018976582301378553

In [92]:
lf = Lasso(fit_intercept=True, alpha=0.5, normalize=True)
lf.fit(X_train_polynomial, y_train)

Lasso(alpha=0.5, copy_X=True, fit_intercept=True, max_iter=1000,
   normalize=True, positive=False, precompute=False, random_state=None,
   selection='cyclic', tol=0.0001, warm_start=False)

In [93]:
lf.score(X_test_polynomial, y_test)

-0.012567943836912843